<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/gen_soil_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score

In [222]:
def preprocess_data(matched_data):
    # Initial preprocessing
    matched_data['distance'] = 0
    matched_data = matched_data[matched_data["depth_adj_bottom"] == '20']
    matched_data['imp_c_float'] = [float(datum) for datum in matched_data['imp_c']]

    # Select ID fields that are non-numeric since can't run minmax scalar over non numeric
    id_fields = matched_data[['source_dataset', 'island','dist_id', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]

    # Select numeric columns and preprocess
    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                 'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                 'agbd_m', 'agbd_sd', 'agbd_n']
    numeric_cols = matched_data[keep_cols]
    numeric_cols.replace('', np.nan, inplace=True)
    numeric_cols = numeric_cols.astype(float)
    numeric_cols.fillna(0, inplace=True)

    # Scale numeric columns
    scaler = MinMaxScaler()
    scaler.fit(numeric_cols)
    scaled_numeric_cols = scaler.transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)

    #min max scale imp_c, keep scalers
    min_c = np.min(matched_data['imp_c_float'])
    max_c = np.max(matched_data['imp_c_float'])
    scaled_imp_c = (matched_data['imp_c_float'] - min_c) / (max_c - min_c)

    # Combine ID fields with scaled numeric data
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)

    numeric_df['imp_c_scaled'] = scaled_imp_c

    return numeric_df, scaler, min_c, max_c

In [ ]:
def train_model_a(preprocess_data):

    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']

    X = preprocess_data[keep_cols]

    y = preprocess_data['imp_c_scaled']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define model
    def build_model(input_shape):
        model = Sequential([
            Dense(256, activation='relu', input_dim=input_shape),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(512, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1, activation='linear')
        ])
        return model

    model = build_model(X_train.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)
    predictions = model.predict(X_test).flatten()
    r_squared = r2_score(y_test, predictions)

    return model, test_loss, r_squared, predictions, y_test

In [4]:
# imputed SOC dostributions
file_path = '/content/drive/MyDrive/hawaii_soils/HI soils data/annotated_combo_imputed_SOC.gpkg'

gdf = gpd.read_file(file_path)

In [310]:
def predict_on_reserve(preprocess_data, model, min_c, max_c):

    df_out = preprocess_data.copy()
    keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
                'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
                'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
                'agbd_m', 'agbd_sd', 'agbd_n']

    X = preprocess_data[keep_cols]
    y = preprocess_data['imp_c_scaled']

    # Predictions and Evaluation
    predictions = model.predict(X)

    inversed_predictions =  predictions*max_c+min_c

    inversed_truth =  y*max_c+min_c

    df_out['predictions'] = predictions
    df_out['inversed_predictions'] = inversed_predictions
    df_out['inversed_imp_c'] = inversed_truth

    return df_out

In [7]:
def evaluate_model(model, X_test, y_test, scaler, inversed_predictions, inversed_truth):
    test_loss = model.evaluate(X_test, y_test)
    r_squared = r2_score(inversed_truth, inversed_predictions)

    print("Test Loss:", test_loss)
    print("R-Squared Score:", r_squared)

    mae = mean_absolute_error(inversed_truth, inversed_predictions)
    rmse = mean_squared_error(inversed_truth, inversed_predictions, squared=False)

    print("Mean Absolute Error (MAE):", mae)
    print("Root Mean Squared Error (RMSE):", rmse)

    # Scatter plot of true vs predicted values
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=inversed_truth, y=inversed_predictions)
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("True vs Predicted Values")
    plt.plot([min(inversed_truth), max(inversed_truth)], [min(inversed_truth), max(inversed_truth)], 'r')  # Diagonal line
    plt.show()

    # Residual plot
    residuals = inversed_truth - inversed_predictions
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel("Residuals")
    plt.title("Distribution of Residuals")
    plt.show()

# Assuming model, test_loss, r_squared, inversed_predictions, inversed_truth, X_test, and y_test are already computed
# model, test_loss, r_squared, inversed_predictions, inversed_truth, scaler, X_test, y_test = preprocess_and_train_model(matched_data)
# evaluate_model(model, X_test, y_test, scaler, inversed_predictions, inversed_truth)




In [9]:
file_path = '/content/drive/My Drive/sequence_data.csv'

In [10]:
drivers_gpd = gpd.read_file('/content/drive/MyDrive/hawaii_soils/Analysis Data/250_summary_grid_dt.gpkg')

In [11]:
soils_csv = gpd.read_file('/content/drive/MyDrive/hawaii_soils/HI soils data/combined_soc_2024_04_05.csv')

In [279]:
# Filter the DataFrame using .loc
soils_csv = soils_csv.loc[(soils_csv['latitude'] != '') & (soils_csv['longitude'] != '')]

# Create geometry column
soils_csv['geometry'] = soils_csv.apply(lambda row: Point(float(row['longitude']), float(row['latitude'])), axis=1)

# Convert to GeoDataFrame
soils_gpd = gpd.GeoDataFrame(soils_csv, geometry='geometry', crs="EPSG:4326")


In [280]:
soils_gpd = pd.merge(soils_gpd, gdf.loc[:,['dist_id','unique_id']], on='unique_id', how='inner')

In [423]:
# Ensure both GeoDataFrames have the same CRS
soils_gpd = soils_gpd.to_crs(drivers_gpd.crs)

# Perform spatial join
matched_data = gpd.sjoin_nearest(soils_gpd, drivers_gpd, how='left', distance_col='distance')

In [424]:
# matches_within_distance = matched_data[matched_data['distance'] <=1000]

In [421]:
# Apply a buffer to each geometry in one of the GeoDataFrames (e.g., soils_gpd)
soils_buffered = soils_gpd.copy()

soils_buffered.geometry = soils_buffered.to_crs(epsg=32604).geometry.buffer(1000).to_crs('ESRI:102261')

# soils_gpd = soils_gpd
# Step 2: Spatial Join
# Perform a spatial join with the buffered GeoDataFrame
# This finds all drivers_gpd points that fall within the 10,000-meter buffer of any point in soils_gpd
matches_within_distance = gpd.sjoin(soils_buffered, drivers_gpd, how='left', predicate='intersects')


In [425]:
matches_within_distance

,index_left,source_dataset,island,soil_column_id,unique_id,depth_top,depth_bottom,depth_adj_bottom,latitude,longitude,...,vpd,vs,agbd_m,agbd_sd,agbd_n,majorcomposition,age_years,age_class,geometry,distance
65,15,FIA,Kauai,FIA_1080,FIA16,0,20,20,22.14805,-159.620943,...,33.527164,414.471527,5.79101,0.412106,2,Tholeiitic Basalt,2000000.0,Holocene,POINT (74661.364 372702.317),34.130651
65,16,FIA,Kauai,FIA_1080,FIA17,40,60,60,22.14805,-159.620943,...,33.527164,414.471527,5.79101,0.412106,2,Tholeiitic Basalt,2000000.0,Holocene,POINT (74661.364 372702.317),34.130651
65,17,FIA,Kauai,FIA_1080,FIA18,60,80,80,22.14805,-159.620943,...,33.527164,414.471527,5.79101,0.412106,2,Tholeiitic Basalt,2000000.0,Holocene,POINT (74661.364 372702.317),34.130651
65,18,FIA,Kauai,FIA_1080,FIA19,80,100,100,22.14805,-159.620943,...,33.527164,414.471527,5.79101,0.412106,2,Tholeiitic Basalt,2000000.0,Holocene,POINT (74661.364 372702.317),34.130651
65,19,FIA,Kauai,FIA_1080,FIA20,20,40,40,22.14805,-159.620943,...,33.527164,414.471527,5.79101,0.412106,2,Tholeiitic Basalt,2000000.0,Holocene,POINT (74661.364 372702.317),34.130651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213192,355,FIA,Hawaii,FIA_2799,FIA356,0,20,20,19.160629,-155.476816,...,88.017952,596.561523,NaN,NaN,0,Alkalic Basalt,300000.0,Holocene,POINT (503411.364 36202.317),973.045832
213192,356,FIA,Hawaii,FIA_2799,FIA357,20,40,40,19.160629,-155.476816,...,88.017952,596.561523,NaN,NaN,0,Alkalic Basalt,300000.0,Holocene,POINT (503411.364 36202.317),973.045832
213192,357,FIA,Hawaii,FIA_2799,FIA358,40,60,60,19.160629,-155.476816,...,88.017952,596.561523,NaN,NaN,0,Alkalic Basalt,300000.0,Holocene,POINT (503411.364 36202.317),973.045832
213192,358,FIA,Hawaii,FIA_2799,FIA359,60,80,80,19.160629,-155.476816,...,88.017952,596.561523,NaN,NaN,0,Alkalic Basalt,300000.0,Holocene,POINT (503411.364 36202.317),973.045832


In [426]:
numeric_df, scaler, min_c, max_c = preprocess_data(matches_within_distance)

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
<ipython-input-222-48d85629ab51>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

array(['20'], dtype=object)

In [288]:
frequency_table = numeric_df['unique_id'].value_counts().reset_index()
frequency_table.columns = ['unique_id', 'count']
frequency_table['count'].unique()

array([52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 38, 37, 36, 35,
       34, 33, 32, 31, 30, 29, 28, 27, 26, 22,  3,  1])

In [225]:
# selecting one random distribution id to leave out for cross validation
xval_id = np.random.choice(numeric_df['dist_id'].unique())

In [228]:
# reserve_data = numeric_df[numeric_df['dist_id']!=xval_id].groupby('unique_id').sample(n=1)
# reserve_data

In [290]:
reserve_data = numeric_df[numeric_df['dist_id']!=xval_id].groupby('unique_id').sample(n=1)
reserve_data

,source_dataset,island,dist_id,soil_column_id,unique_id,depth_top,depth_bottom,depth_adj_bottom,latitude,longitude,...,swe,tmmn,tmmx,vap,vpd,vs,agbd_m,agbd_sd,agbd_n,imp_c_scaled
0,FIA,Kauai,NaN,FIA_1060,FIA1,0,20,20,22.185643,-159.350787,...,0.0,0.919650,0.946761,0.947442,0.833200,0.510523,0.016828,0.030577,0.137931,0.064231
100,FIA,Oahu,NaN,FIA_1433,FIA101,0,20,20,21.475448,-157.984458,...,0.0,0.900208,0.934330,0.924736,0.816007,0.514500,0.094650,0.164560,0.551724,0.064954
105,FIA,Oahu,NaN,FIA_1454,FIA106,0,20,20,21.419936,-157.906723,...,0.0,0.871936,0.909543,0.898124,0.762932,0.542160,0.044217,0.070038,0.465517,0.143556
10,FIA,Kauai,NaN,FIA_1077,FIA11,0,20,20,22.152306,-159.380618,...,0.0,0.818287,0.885699,0.885619,0.657210,0.487529,0.205160,0.517108,0.086207,0.099888
110,FIA,Oahu,NaN,FIA_1457,FIA111,0,20,20,21.404189,-157.854121,...,0.0,0.876864,0.910243,0.898774,0.769978,0.544770,0.000000,0.000000,0.000000,0.039570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,SH,Kauai,NaN,SH94-01,SH95,0,15,20,21.967096,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.004020,0.007715,0.086207,0.113633
5998,SH,Kauai,NaN,SH94-01,SH96,0,15,20,21.967096,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.002663,0.008040,0.103448,0.076609
5999,SH,Kauai,NaN,SH97-01,SH97,0,15,20,21.9670959,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.006749,0.009302,0.137931,0.107264
6000,SH,Kauai,NaN,SH97-01,SH98,0,15,20,21.9670959,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.000306,0.000000,0.017241,0.025221


In [291]:
reserve_data = numeric_df[numeric_df['dist_id']!=xval_id].groupby('unique_id').sample(n=1)
reserve_data

,source_dataset,island,dist_id,soil_column_id,unique_id,depth_top,depth_bottom,depth_adj_bottom,latitude,longitude,...,swe,tmmn,tmmx,vap,vpd,vs,agbd_m,agbd_sd,agbd_n,imp_c_scaled
0,FIA,Kauai,NaN,FIA_1060,FIA1,0,20,20,22.185643,-159.350787,...,0.0,0.919650,0.946761,0.947442,0.833200,0.510523,0.049059,0.124622,0.068966,0.064231
100,FIA,Oahu,NaN,FIA_1433,FIA101,0,20,20,21.475448,-157.984458,...,0.0,0.900208,0.934330,0.924736,0.816007,0.514500,0.078824,0.148215,0.241379,0.064954
105,FIA,Oahu,NaN,FIA_1454,FIA106,0,20,20,21.419936,-157.906723,...,0.0,0.887293,0.920565,0.910170,0.790353,0.539879,0.039453,0.044276,0.379310,0.143556
10,FIA,Kauai,NaN,FIA_1077,FIA11,0,20,20,22.152306,-159.380618,...,0.0,0.849930,0.904856,0.904350,0.711917,0.491875,0.126932,0.231857,0.051724,0.099888
110,FIA,Oahu,NaN,FIA_1457,FIA111,0,20,20,21.404189,-157.854121,...,0.0,0.876864,0.910243,0.898774,0.769978,0.544770,0.030351,0.065481,0.137931,0.039570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5997,SH,Kauai,NaN,SH94-01,SH95,0,15,20,21.967096,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.002138,0.002781,0.068966,0.113633
5998,SH,Kauai,NaN,SH94-01,SH96,0,15,20,21.967096,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.000651,0.002159,0.137931,0.076609
5999,SH,Kauai,NaN,SH97-01,SH97,0,15,20,21.9670959,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.003482,0.014753,0.172414,0.107264
6000,SH,Kauai,NaN,SH97-01,SH98,0,15,20,21.9670959,-159.692819,...,0.0,0.902048,0.954855,0.950564,0.826877,0.519562,0.000000,0.000000,0.000000,0.025221


In [292]:
# numeric_df[numeric_df['unique_id'] == 'FIA1']

In [311]:
# Use the function with your DataFrame
reserve_data = numeric_df[numeric_df['dist_id']!=xval_id].groupby('unique_id').sample(n=1)
r2_list = []
loss_list = []
prediction_list = []
matched_data_list = []
model_list = []
for i in range(10):
  matched_data = numeric_df[numeric_df['dist_id']!=xval_id].groupby('unique_id').sample(n=1)
  matched_data = matched_data.reset_index(drop=True)
  model, test_loss, r_squared, predictions, y_test = train_model_a(matched_data)
  predictions = predict_on_reserve(reserve_data, model, min_c, max_c)
  matched_data_list.append(matched_data)
  prediction_list.append(predictions)
  model_list.append(model)

Epoch 1/100
14/14 [==============================] - 2s 19ms/step - loss: 2.1307 - val_loss: 0.0307
Epoch 2/100
14/14 [==============================] - 0s 7ms/step - loss: 1.6448 - val_loss: 0.0204
Epoch 3/100
14/14 [==============================] - 0s 7ms/step - loss: 1.1009 - val_loss: 0.0207
Epoch 4/100
14/14 [==============================] - 0s 7ms/step - loss: 0.9411 - val_loss: 0.0217
Epoch 5/100
14/14 [==============================] - 0s 7ms/step - loss: 0.7457 - val_loss: 0.0182
Epoch 6/100
14/14 [==============================] - 0s 6ms/step - loss: 0.6461 - val_loss: 0.0187
Epoch 7/100
14/14 [==============================] - 0s 7ms/step - loss: 0.5693 - val_loss: 0.0214
Epoch 8/100
14/14 [==============================] - 0s 7ms/step - loss: 0.4610 - val_loss: 0.0196
Epoch 9/100
14/14 [==============================] - 0s 7ms/step - loss: 0.4031 - val_loss: 0.0213
Epoch 10/100
14/14 [==============================] - 0s 7ms/step - loss: 0.3374 - val_loss: 0.0178
Epoch 11

In [312]:

# X = preprocess_data[keep_cols]

# [model.predict(reserve_data[keep_cols]) for model in model_list]

In [313]:
[np.all(prediction['predictions'] ==prediction_list[0]['predictions']) for prediction in prediction_list]

[True, False, False, False, False, False, False, False, False, False]

In [314]:
# prediction_list[0]

In [315]:
combined_array = np.array([predictions['predictions'].values for predictions in prediction_list])

In [316]:
combined_array[combined_array < 0 ] = 0

In [198]:
# combined_array[0]

In [317]:
arr_min = np.min(combined_array.flatten())
arr_max = np.max(combined_array.flatten())

In [318]:
arr_max

0.4143471

In [319]:
dist_array = [np.sort(np.array([i[j] for  i in combined_array])) for j in range(combined_array.shape[1])]
norm_dist_array =  [np.sort((np.array([i[j] for  i in combined_array]) - arr_min) / (arr_max-arr_min)) for j in range(combined_array.shape[1])]

In [324]:
# norm_dist_array

In [321]:
array = norm_dist_array[1]
# Generating y values as indices (percentiles in this context)
y_values = np.linspace(1, 100, len(array))

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(array, y_values, marker='o', linestyle='-', color='b')
plt.title('Index (Percentile) by Value')
plt.xlabel('Value')
plt.ylabel('Percentile')
plt.grid(True)
plt.show()

# Plotting
plt.figure(figsize=(10, 6))
plt.hist(array, bins=len(array), edgecolor='black')
plt.title('Histogram of Values')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()



need to check the below code block to see if "predictions" is the right data to reference (was previously coded as pred_values)

In [328]:
keep_cols = ['water', 'trees','grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built',
            'bare', 'snow_and_ice', 'max', 'elevation', 'landform', 'SRTM_mTPI','aet', 'def',
            'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn','tmmx', 'vap', 'vpd', 'vs',
            'agbd_m', 'agbd_sd', 'agbd_n']

scaled_numeric_df = prediction_list[0][keep_cols]
# numeric_cols = predictions.iloc[:,keep_cols]

scaled_numeric_df['norm_dist_array'] = norm_dist_array

# id_fields = predictions[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]

# numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)


<ipython-input-328-eadb985067ec>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_numeric_df['norm_dist_array'] = norm_dist_array


In [ ]:
numeric_df.columns

In [ ]:
# Model Definition
def build_model(input_shape,output_shape):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_shape),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(output_shape, activation='linear')  # Adjust output layer according to your needs
    ])
    return model


In [329]:
scaled_numeric_df['norm_dist_array'] = scaled_numeric_df['norm_dist_array'].to_list()

<ipython-input-329-4a7713e6d101>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_numeric_df['norm_dist_array'] = scaled_numeric_df['norm_dist_array'].to_list()


In [330]:
scaled_numeric_df.dropna(inplace=True)

<ipython-input-330-8f639aa31ceb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_numeric_df.dropna(inplace=True)


In [331]:
norm_dist_array = np.array([i for i in scaled_numeric_df['norm_dist_array']])

In [332]:

X = scaled_numeric_df.iloc[:, :-1]
y = scaled_numeric_df.iloc[:, -1]

x = np.array(X)
y = np.array(norm_dist_array)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [333]:
# print(X_train.shape[1])
model = build_model(X_train.shape[1],output_shape=10)

In [334]:
y_train.shape

(2172, 10)

In [335]:
# model = build_model((X_train.shape[1]))
# model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error',metrics = ['mae'])
# model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)

test_loss = model.evaluate(X_test, y_test)

# Predictions and Evaluation
predictions = model.predict(X_test)

Epoch 1/600
14/14 [==============================] - 3s 28ms/step - loss: 1.9950 - mae: 1.0541 - val_loss: 0.0755 - val_mae: 0.2090
Epoch 2/600
14/14 [==============================] - 0s 8ms/step - loss: 1.6860 - mae: 0.9711 - val_loss: 0.0741 - val_mae: 0.2054
Epoch 3/600
14/14 [==============================] - 0s 7ms/step - loss: 1.5690 - mae: 0.9272 - val_loss: 0.0757 - val_mae: 0.2093
Epoch 4/600
14/14 [==============================] - 0s 7ms/step - loss: 1.4295 - mae: 0.8889 - val_loss: 0.0772 - val_mae: 0.2125
Epoch 5/600
14/14 [==============================] - 0s 7ms/step - loss: 1.3308 - mae: 0.8556 - val_loss: 0.0780 - val_mae: 0.2145
Epoch 6/600
14/14 [==============================] - 0s 7ms/step - loss: 1.2472 - mae: 0.8322 - val_loss: 0.0786 - val_mae: 0.2160
Epoch 7/600
14/14 [==============================] - 0s 7ms/step - loss: 1.2096 - mae: 0.8182 - val_loss: 0.0791 - val_mae: 0.2170
Epoch 8/600
14/14 [==============================] - 0s 7ms/step - loss: 1.1892 - 

In [336]:
i = 4
for i in range(5):
  print(predictions[i])
  print(y_test[i])

[0.48115852 0.5277555  0.5496578  0.56039125 0.58340293 0.59936875
 0.6320796  0.6476236  0.66719    0.69701815]
[0.48377988 0.5139347  0.5322559  0.55730677 0.59377635 0.61040044
 0.65095294 0.6788908  0.7008108  0.7021959 ]
[-0.01941803 -0.01167763 -0.00271242  0.00630163  0.00987469  0.0176178
  0.02669702  0.03404857  0.05862647  0.08836857]
[0.         0.         0.         0.         0.00416696 0.01195159
 0.01502147 0.03817881 0.11299475 0.11752209]
[0.01520755 0.04215664 0.06175364 0.06800167 0.07660041 0.08736277
 0.10085265 0.11261874 0.1353065  0.16194686]
[0.00234122 0.03538123 0.05592024 0.06191235 0.07766561 0.08460186
 0.10648334 0.11831888 0.1443978  0.17861046]
[-0.01007077 -0.01235103  0.00627944  0.01453763  0.02232897  0.0288805
  0.05321099  0.0550379   0.05723338  0.08213631]
[0.         0.         0.0197088  0.02938866 0.03878339 0.04625907
 0.07370926 0.077751   0.08210102 0.10831274]
[0.02904916 0.05076142 0.06521464 0.07079199 0.07609979 0.08416987
 0.09259874

In [441]:
xval_data = numeric_df[numeric_df['dist_id']==xval_id]

In [442]:
xval_predictions = model.predict(xval_data[keep_cols])

1/1 [==============================] - 0s 18ms/step


In [444]:
xval_data

,source_dataset,island,dist_id,soil_column_id,unique_id,depth_top,depth_bottom,depth_adj_bottom,latitude,longitude,...,swe,tmmn,tmmx,vap,vpd,vs,agbd_m,agbd_sd,agbd_n,imp_c_scaled
79482,HISOC,Maui,30.0,HISOC358,HISOC1356,0,15,20,20.757154,-156.267174,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.008830,0.051573,0.310345,0.183533
79482,HISOC,Maui,30.0,HISOC358,HISOC1357,15,30,20,20.757154,-156.267174,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.008830,0.051573,0.310345,0.071849
79483,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.002243,0.005848,0.137931,0.229442
79483,HISOC,Maui,30.0,HISOC354,HISOC1337,15,30,20,20.7567,-156.26802,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.002243,0.005848,0.137931,0.160302
79484,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.005309,0.006983,0.103448,0.229442
79484,HISOC,Maui,30.0,HISOC354,HISOC1337,15,30,20,20.7567,-156.26802,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.005309,0.006983,0.103448,0.160302
79485,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.512509,0.528467,0.469577,0.207457,0.398008,0.004221,0.008886,0.258621,0.229442
79485,HISOC,Maui,30.0,HISOC354,HISOC1337,15,30,20,20.7567,-156.26802,...,0.0,0.512509,0.528467,0.469577,0.207457,0.398008,0.004221,0.008886,0.258621,0.160302
79487,HISOC,Maui,30.0,HISOC358,HISOC1356,0,15,20,20.757154,-156.267174,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.006827,0.045046,0.413793,0.183533
79487,HISOC,Maui,30.0,HISOC358,HISOC1357,15,30,20,20.757154,-156.267174,...,0.0,0.516819,0.532400,0.473717,0.210099,0.394670,0.006827,0.045046,0.413793,0.071849


In [433]:
xval_norm_predictions = [prediction*max_c+min_c for prediction in xval_predictions]

In [434]:
xval_quantiles = np.quantile((xval_data['imp_c_scaled']*max_c+min_c).unique(), np.linspace(0, 1,10))

In [435]:
[np.mean(np.sort(prediction) - np.sort(xval_quantiles)) for prediction in xval_norm_predictions]

[7.61766519174996,
 7.61766519174996,
 7.136256786446003,
 7.136256786446003,
 6.353644271950154,
 6.353644271950154,
 6.852491565803914,
 6.852491565803914,
 6.752681919197468,
 6.752681919197468,
 5.555017562965779,
 5.555017562965779,
 5.867695518593221,
 5.867695518593221,
 6.827581592659382,
 6.827581592659382,
 8.077310176948934,
 8.077310176948934,
 5.798564525703816,
 5.798564525703816,
 9.267343326668172,
 9.267343326668172,
 7.3087887726825755,
 7.3087887726825755]

In [440]:
# xval_data

In [437]:
xval_data['mean_resid']= [np.mean(np.sort(prediction) - np.sort(xval_quantiles)) for prediction in xval_norm_predictions]

<ipython-input-437-c3ba60135f41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xval_data['mean_resid']= [np.mean(np.sort(prediction) - np.sort(xval_quantiles)) for prediction in xval_norm_predictions]


In [438]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

xval_data['geometry'] = [Point(xy) for xy in zip(xval_data['longitude'], xval_data['latitude'])]

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(xval_data, geometry='geometry')

# Set the coordinate reference system (CRS) for the GeoDataFrame
gdf.set_crs(epsg=4326, inplace=True)  # EPSG 4326 - WGS84 latitude and longitude

gdf.to_file('test.gpkg')

<ipython-input-438-53bf293a6d19>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xval_data['geometry'] = [Point(xy) for xy in zip(xval_data['longitude'], xval_data['latitude'])]


In [337]:
def normal_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return np.random.normal(variable_mean, variable_sd, n)

def uniform_generator(n, min_val, max_val, min_variation, max_variation):
    variable_min = min_val + np.random.uniform(-min_variation * 2, min_variation * 2)
    variable_max = max_val + np.random.uniform(-max_variation * 2, max_variation * 2)
    return np.random.uniform(variable_min, variable_max, n)

def right_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return np.random.lognormal(variable_mean, variable_sd, n)

def left_tailed_generator(n, mean, sd, mean_variation, sd_variation):
    variable_mean = mean + np.random.normal(0, mean_variation * 2)
    variable_sd = sd + abs(np.random.normal(0, sd_variation * 2))
    return -np.random.lognormal(variable_mean, variable_sd, n)

def multimodal_generator(ns, means, sds, mean_variation, sd_variation):
    samples_list = []
    for i in range(len(means)):
        variable_mean = means[i] + np.random.normal(0, mean_variation * 2)
        variable_sd = sds[i] + abs(np.random.normal(0, sd_variation * 2))
        samples_list.append(np.random.normal(variable_mean, variable_sd, ns[i]))
    return np.concatenate(samples_list)

In [338]:
def rescale_to_minus_one_one(values):
    min_value = np.min(values)
    max_value = np.max(values)
    scaled_values = 2 * ((values - min_value) / (max_value - min_value)) - 1
    return scaled_values

In [339]:
def generate_samples(n, q, generator_func, *args):
    samples = generator_func(n, *args)
    samples_q = np.quantile(samples, np.linspace(0, 1, q))
    return rescale_to_minus_one_one(samples_q)

def generate_normal_samples(n=10000, q=10, mean_value=0, sd_value=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, normal_generator, mean_value, sd_value, mean_variation, sd_variation)

def generate_uniform_samples(n=10000, q=10, min_value=-1, max_value=1, min_variation=0.5, max_variation=0.5):
    return generate_samples(n, q, uniform_generator, min_value, max_value, min_variation, max_variation)

def generate_right_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, right_tailed_generator, mean, sd, mean_variation, sd_variation)

def generate_left_tailed_samples(n=10000, q=10, mean=0, sd=1, mean_variation=0.5, sd_variation=0.5):
    return generate_samples(n, q, left_tailed_generator, mean, sd, mean_variation, sd_variation)

def generate_multimodal_samples(ns=[1000, 50], q=10, means=[1, -1], sds=[1, 0.5], mean_variation=0.5, sd_variation=0.5):
    samples = multimodal_generator(ns, means, sds, mean_variation, sd_variation)
    samples_q = np.quantile(samples, np.linspace(0, 1, q))
    return rescale_to_minus_one_one(samples_q)


In [340]:
def generate_training_data(n_samples=10000):
    data_list = {}

    # Generate samples for each distribution type
    data_list['normal'] = [generate_normal_samples() for _ in range(n_samples)]
    data_list['bimodal'] = [generate_multimodal_samples() for _ in range(n_samples)]
    data_list['uniform'] = [generate_uniform_samples() for _ in range(n_samples)]
    data_list['right_tailed'] = [generate_right_tailed_samples() for _ in range(n_samples)]
    data_list['left_tailed'] = [generate_left_tailed_samples() for _ in range(n_samples)]

    # Combine all data into a single DataFrame
    combined_data = pd.DataFrame()
    for name, samples_list in data_list.items():
        df = pd.DataFrame(samples_list)
        df['label'] = name
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    # Assign column names
    combined_data.columns = [f'V{i+1}' for i in range(combined_data.shape[1] - 1)] + ['label']

    return combined_data

In [341]:
training_data = generate_training_data()

In [342]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [343]:
# Separate features and labels
X = training_data.drop(columns=['label'])
y = training_data['label']

# Encode labels as integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=50, min_samples_leaf=2, random_state=123)

# Train the model
rf_model.fit(X, y_encoded)

# Print the trained model
print(rf_model)

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2, n_estimators=500,
                       random_state=123)


In [344]:
reference_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)) for prediction in  y_test]

In [345]:
# reference_rf_ready_predictions

In [346]:
some_new_data = training_data = generate_training_data()

In [347]:
# predictions

In [348]:
prediction_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)) for prediction in  predictions]

In [357]:
xval_data

,source_dataset,island,dist_id,soil_column_id,unique_id,depth_top,depth_bottom,depth_adj_bottom,latitude,longitude,...,swe,tmmn,tmmx,vap,vpd,vs,agbd_m,agbd_sd,agbd_n,imp_c_scaled
1765,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.337420,0.552525,0.497723,0.306357,0.749331,0.009111,0.034118,0.568966,0.229442
1765,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.337420,0.552525,0.497723,0.306357,0.749331,0.003574,0.007104,0.258621,0.229442
1765,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.337420,0.552525,0.497723,0.306357,0.749331,0.006506,0.025007,0.344828,0.229442
1765,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.412727,0.601235,0.549741,0.341944,0.717978,0.007514,0.028901,0.362069,0.229442
1765,HISOC,Maui,30.0,HISOC354,HISOC1336,0,15,20,20.7567,-156.26802,...,0.0,0.337420,0.552525,0.497723,0.306357,0.749331,0.004731,0.010403,0.224138,0.229442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,HISOC,Maui,30.0,HISOC358,HISOC1357,15,30,20,20.757154,-156.267174,...,0.0,0.516819,0.668563,0.621640,0.391135,0.674640,0.001790,0.005487,0.413793,0.071849
1786,HISOC,Maui,30.0,HISOC358,HISOC1357,15,30,20,20.757154,-156.267174,...,0.0,0.516819,0.668563,0.621640,0.391135,0.674640,0.001485,0.002480,0.224138,0.071849
1786,HISOC,Maui,30.0,HISOC358,HISOC1357,15,30,20,20.757154,-156.267174,...,0.0,0.516819,0.668563,0.621640,0.391135,0.674640,0.008482,0.041357,0.327586,0.071849
1786,HISOC,Maui,30.0,HISOC358,HISOC1357,15,30,20,20.757154,-156.267174,...,0.0,0.516819,0.668563,0.621640,0.391135,0.674640,0.008171,0.041527,0.327586,0.071849


In [349]:
real_quantiles.columns = [f'V{i+1}' for i in range(real_quantiles.shape[1])]

NameError: name 'real_quantiles' is not defined

In [352]:
real_data_rf_ready_predictions = [rescale_to_minus_one_one(sorted(prediction)) for prediction in  np.array(real_quantiles) ]

NameError: name 'real_quantiles' is not defined

In [353]:
# Assuming real_data_rf_ready_predictions is your list of predictions
real_data_rf_ready_predictions = [prediction for prediction in real_data_rf_ready_predictions if not np.any(np.isnan(prediction))]


NameError: name 'real_data_rf_ready_predictions' is not defined

In [354]:
real_data_rf_ready_predictions

NameError: name 'real_data_rf_ready_predictions' is not defined

In [355]:
real_quantiles = quantiles.unstack().reset_index().iloc[:,1:]

ValueError: index must be a MultiIndex to unstack, <class 'pandas.core.indexes.base.Index'> was passed

In [ ]:
real_data_rf_ready_predictions

In [ ]:
# Assuming `new_data` is your new dataset without the 'label' column

# Use the trained model to make predictions
predicted_labels_encoded = rf_model.predict(real_data_rf_ready_predictions)

# Decode the encoded labels back to original labels
predicted_predicted_labels = label_encoder.inverse_transform(predicted_labels_encoded)

# Output the predictions
print(predicted_predicted_labels)


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(predicted_predicted_labels, reference_predicted_labels)


In [ ]:
cm

In [ ]:
import numpy as np
import pandas as pd

# Assuming 'my_array' is your numpy array
unique_values, counts = np.unique(predicted_predicted_labels, return_counts=True)

# Create a DataFrame to display the counts in a table format
counts_table = pd.DataFrame({
    'Unique Value': unique_values,
    'Count': counts
})

# Display the table
print(counts_table)


In [ ]:
import numpy as np
import pandas as pd

# Assuming 'my_array' is your numpy array
unique_values, counts = np.unique(predicted_labels, return_counts=True)

# Create a DataFrame to display the counts in a table format
counts_table = pd.DataFrame({
    'Unique Value': unique_values,
    'Count': counts
})

# Display the table
print(counts_table)


In [ ]:
# matches_within_distance[matches_within_distance['dist_id']!=xval_id].groupby('unique_id').sample(n=1)

In [ ]:
xval_data = matches_within_distance[matches_within_distance['dist_id']==xval_id]

In [ ]:
quantiles = xval_data[xval_data['depth_adj_bottom'] =="20" ].groupby('unique_id').sample(n=1)['imp_c'].astype(float).quantile(np.linspace(0, 1,10))

In [ ]:
keep_cols = [21,22,23,24,25,26,27,28, 29, 31,32, 33,34,36,36,37,38,39,40,41,42,43,44,45,46,47, 48]

numeric_cols = xval_data.iloc[:,keep_cols]

# scaler = MinMaxScaler()

scaler.fit(numeric_cols)

scaled_numeric_cols = scaler.transform(numeric_cols)

scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)

# scaled_numeric_df['norm_dist_array'] = norm_dist_array

id_fields = xval_data[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]

numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)

In [ ]:
X = scaled_numeric_df.iloc[:, :-1]


In [ ]:
predictions = model.predict(scaled_numeric_df)

In [ ]:
predictions

In [ ]:
dummy_array = np.full((len(predictions), 28), fill_value=0.5)  # Example for reverse scaling
dummy_array[:, -1] = predictions
inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]
dummy_array[:, -1] = y_test
inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]